## Format App Data

Format data for direct App ingest:
* Results data table
* Plot color conventions
* Geneset dictionary

## Set Up

In [1]:
# pip install session_info

In [2]:
import pandas as pd
from warnings import warn
import numpy as np
import hisepy as hp
import os
import session_info
import pickle
import re

## Functions

In [3]:
def check_cat_dict(cat_dict, df, verbose = False):
    ''' 
    Checks that a dictionary of column names and ordered expected values represents complete 
    data given the data frame.
    Requires: from warnings import warn
    '''
    
    for name in cat_dict.keys():
        if(name not in df.columns):
            warn('category dictionary column name not found in data: ' + name)
        else:
            data_cat_all = pd.unique(df[name])
            missing_cat = [x for x in data_cat_all if x not in cat_dict[name]]
            if len(missing_cat) > 0:
                warn('The following values in data column "' + name + '" are not found in category dictionary: ' + ", ".join(missing_cat) + 
                     ". Add to dictionary if you wish to include as categorical levels for plot filtering, etc.")
            extra_cat = [x for x in cat_dict[name] if x not in data_cat_all]
            if len(extra_cat) > 0:
                warn('Extra data levels found in data dictionary for column "' + name + '": ' + ", ".join(extra_cat) + 
                         ". These will be removed as categories during variable formatting.")
    if(verbose):
        print("Finished checking category dictionary")

# #test1: bad dictionary colname
# test1_dict = {'cell_type':ct_order,
#            'randomcol': [1,2,4,23,8],
#           'experiment':expt_order}
# check_cat_dict(test1_dict, df, verbose = True)

# #test2: missing data level
# test2_dict = {'cell_type': ct_order[0:10],
#               'experiment':expt_order}
# check_cat_dict(test2_dict, df, verbose = True)


# #test3: bad dictionary colname and missing data
# test3_dict = {'cell_type':ct_order[0:10],
#                'randomcol': [1,2,4,23,8],
#               'experiment':expt_order}
# check_cat_dict(test3_dict, df, verbose = True)

# #test4: extra category
# test_order = ct_order[0:10].copy()
# test_order.extend(['random_celltype'])
# test4_dict = {'cell_type': test_order,
#               'randomcol': [1,2,4,23,8],
#               'experiment':expt_order}
# check_cat_dict(test4_dict, df, verbose = True)

# check_cat_dict(test_dict, test_df, verbose=True)

In [4]:
def format_df(df, cat_dict={}):
    ''' 
    Converts all string columns to categorical, using input dictionary to set 
    value order if supplied, otherwise sets order as alphanumeric sorted values
    
    Requires: import pandas as pd; import numpy as np
    '''
    res = df.copy()
    
    # Formats
    types = [res[x].dtype for x in res.columns]
    
    # Convert string to categorical
    i_str = [i for i in range(len(types)) if types[i] =='O']
    if(len(i_str) > 0):
        if(len(i_str) > 0):
            cnames = [res.columns[i] for i in i_str]
            cat_dict_all = {x:cat_dict[x] if x in cat_dict.keys() else np.sort(pd.unique(res[x])) for x in cnames}
            for col in cnames:
                res[col] = res[col].astype(pd.CategoricalDtype(categories=cat_dict_all[col], ordered = True))
    return res

In [5]:
def make_comp_label(treatment, timepoint):
    res_str = "TEA-seq " + treatment[0].upper() + treatment[1:3] + ". " + str(int(timepoint)) + " hr"
    return(res_str)


## Read Data

updated data 9/6/23:

In [6]:
# DEG Results
fid ='fc83b89f-fd26-43b8-ac91-29c539703a45'
fd = hp.read_files(file_list = [fid])
os.rename("cache/unknown/all_mast_deg_2023-09-06.csv", "data/all_mast_deg_2023-09-06.csv")

In [7]:
fp = './data/all_mast_deg_2023-09-06.csv'
df = pd.read_csv(fp)
df.head()

,aifi_cell_type,timepoint,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP
0,t_cd4_cm,4,bortezomib,dmso,648,A1BG-AS1,0.028412,-0.127145,-0.008604,0.740370,0.999237
1,t_cd4_cm,4,bortezomib,dmso,648,AAGAB,0.032287,-0.255750,-0.054295,0.145599,0.991887
2,t_cd4_cm,4,bortezomib,dmso,648,AAK1,-0.009058,0.047696,0.014816,0.876254,0.999237
3,t_cd4_cm,4,bortezomib,dmso,648,AAMDC,0.062295,0.192507,0.043135,0.160231,0.991887
4,t_cd4_cm,4,bortezomib,dmso,648,AAMP,0.018081,-0.166733,-0.010012,0.733253,0.999237


In [8]:
df.shape

(390410, 11)

## Format Data

### Add treatment column

In [9]:
df['treatment'] = df.loc[:,'fg']

### Add comparison column

In [10]:
df['comparison'] = df.apply(lambda row: make_comp_label(row.treatment, row.timepoint), axis = 1)

In [11]:
df['comparison'].unique()

array(['TEA-seq Bor. 4 hr', 'TEA-seq Bor. 24 hr', 'TEA-seq Bor. 72 hr',
       'TEA-seq Dex. 4 hr', 'TEA-seq Dex. 24 hr', 'TEA-seq Len. 4 hr',
       'TEA-seq Len. 24 hr', 'TEA-seq Len. 72 hr'], dtype=object)

### Add celltype label column

In [12]:
ct_df = {
    'aifi_cell_type': [
        't_cd4_naive',
        't_cd4_cm',
        't_cd4_em',
        't_cd4_treg',
        't_cd8_naive',
        't_cd8_memory'
    ],
    'cell_type': [
        'CD4 Naive',
        'CD4 CM',
        'CD4 EM',
        'CD4 Treg',
        'CD8 Naive',
        'CD8 Memory'
    ]
}
ct_labs = pd.DataFrame(ct_df)

In [13]:
df = df.merge(ct_labs.loc[:,['aifi_cell_type','cell_type']], on='aifi_cell_type', how = 'left', copy=True)

In [14]:
df['cell_type'].unique()

array(['CD4 CM', 'CD4 EM', 'CD4 Naive', 'CD4 Treg', 'CD8 Memory',
       'CD8 Naive'], dtype=object)

### Check

In [15]:
df.shape

(390410, 14)

In [16]:
df.columns

Index(['aifi_cell_type', 'timepoint', 'fg', 'bg', 'n_sample', 'gene', 'coef_C',
       'coef_D', 'logFC', 'nomP', 'adjP', 'treatment', 'comparison',
       'cell_type'],
      dtype='object')

In [17]:
df.head()

,aifi_cell_type,timepoint,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP,treatment,comparison,cell_type
0,t_cd4_cm,4,bortezomib,dmso,648,A1BG-AS1,0.028412,-0.127145,-0.008604,0.740370,0.999237,bortezomib,TEA-seq Bor. 4 hr,CD4 CM
1,t_cd4_cm,4,bortezomib,dmso,648,AAGAB,0.032287,-0.255750,-0.054295,0.145599,0.991887,bortezomib,TEA-seq Bor. 4 hr,CD4 CM
2,t_cd4_cm,4,bortezomib,dmso,648,AAK1,-0.009058,0.047696,0.014816,0.876254,0.999237,bortezomib,TEA-seq Bor. 4 hr,CD4 CM
3,t_cd4_cm,4,bortezomib,dmso,648,AAMDC,0.062295,0.192507,0.043135,0.160231,0.991887,bortezomib,TEA-seq Bor. 4 hr,CD4 CM
4,t_cd4_cm,4,bortezomib,dmso,648,AAMP,0.018081,-0.166733,-0.010012,0.733253,0.999237,bortezomib,TEA-seq Bor. 4 hr,CD4 CM


In [18]:
df.tail()

,aifi_cell_type,timepoint,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP,treatment,comparison,cell_type
390405,t_cd8_naive,72,lenalidomide,dmso,451,ZUP1,0.020741,-0.127805,-0.013134,0.804388,0.979982,lenalidomide,TEA-seq Len. 72 hr,CD8 Naive
390406,t_cd8_naive,72,lenalidomide,dmso,451,ZW10,0.096564,0.148373,0.018932,0.404856,0.902745,lenalidomide,TEA-seq Len. 72 hr,CD8 Naive
390407,t_cd8_naive,72,lenalidomide,dmso,451,ZXDC,0.031866,-0.011404,0.006514,0.697405,0.962611,lenalidomide,TEA-seq Len. 72 hr,CD8 Naive
390408,t_cd8_naive,72,lenalidomide,dmso,451,ZYG11B,-0.072328,0.125495,0.013986,0.234216,0.839012,lenalidomide,TEA-seq Len. 72 hr,CD8 Naive
390409,t_cd8_naive,72,lenalidomide,dmso,451,ZZEF1,-0.030105,0.027187,-0.001044,0.697492,0.962611,lenalidomide,TEA-seq Len. 72 hr,CD8 Naive


### Variable Levels

Make Categorical data dictionary to explicitly level categorical data. run the `check_cat_dict()` function to check for potential errors

In [19]:
ct_order = ['CD4 Naive','CD4 CM','CD4 EM','CD4 Treg',
            'CD8 Naive', 'CD8 Memory']
comp_order = ['TEA-seq Bor. 4 hr','TEA-seq Bor. 24 hr','TEA-seq Bor. 72 hr',
              'TEA-seq Len. 4 hr','TEA-seq Len. 24 hr','TEA-seq Len. 72 hr',
              'TEA-seq Dex. 4 hr','TEA-seq Dex. 24 hr']
treat_order = ['bortezomib','lenalidomide','dexamethasone']

In [20]:
cat_dict={'cell_type':ct_order,
          'comparison':comp_order,
          'treatment':treat_order}

In [21]:
check_cat_dict(cat_dict, df, verbose = True)

Finished checking category dictionary


In [22]:
%%time
df = format_df(df, cat_dict)

CPU times: user 408 ms, sys: 64.1 ms, total: 472 ms
Wall time: 471 ms


### Save Formatted Data

In [23]:
handle_col = open('data/all_mast_deg_2023-09-06.pkl', 'wb')
pickle.dump(df, handle_col, protocol=pickle.HIGHEST_PROTOCOL)
handle_col.close()

## Color dictionary

In [24]:
color_dict_ct = {
    'CD4 Naive': '#93A7D1',
    'CD4 CM': '#00AEEF',
    'CD4 EM': '#1C75BC',
    'CD4 Treg': '#E57A93',
    'CD8 Naive': '#8DC63F',
    'CD8 Memory': '#009444'
} 

In [25]:
treat_col_dict = {
    'bortezomib': '#FF9F45',
    'lenalidomide': '#59C1BD',
    'dexamethasone': '#8D9EFF'
}

In [26]:
comp_dict = {
    'TEA-seq Len. 72 hr': '#4F8886',
    'TEA-seq Bor. 72 hr': '#F76E11',
    'TEA-seq Dex. 24 hr': '#8D9EFF',
    'TEA-seq Len. 24 hr': '#59C1BD',
    'TEA-seq Bor. 24 hr': '#FF9F45',
    'TEA-seq Dex. 4 hr': '#B9E0FF',
    'TEA-seq Len. 4 hr': '#CFF5E7',
    'TEA-seq Bor. 4 hr': '#FFBC80'
}

In [27]:
preset_colors_dict = {'cell_type':color_dict_ct,
                     'comparison':comp_dict,
                     'treatment':treat_col_dict}
preset_colors_dict

{'cell_type': {'CD4 Naive': '#93A7D1',
  'CD4 CM': '#00AEEF',
  'CD4 EM': '#1C75BC',
  'CD4 Treg': '#E57A93',
  'CD8 Naive': '#8DC63F',
  'CD8 Memory': '#009444'},
 'comparison': {'TEA-seq Len. 72 hr': '#4F8886',
  'TEA-seq Bor. 72 hr': '#F76E11',
  'TEA-seq Dex. 24 hr': '#8D9EFF',
  'TEA-seq Len. 24 hr': '#59C1BD',
  'TEA-seq Bor. 24 hr': '#FF9F45',
  'TEA-seq Dex. 4 hr': '#B9E0FF',
  'TEA-seq Len. 4 hr': '#CFF5E7',
  'TEA-seq Bor. 4 hr': '#FFBC80'},
 'treatment': {'bortezomib': '#FF9F45',
  'lenalidomide': '#59C1BD',
  'dexamethasone': '#8D9EFF'}}

### Save color dictionary

In [28]:
handle_col = open('data/meta_color_dict.pkl', 'wb')
pickle.dump(preset_colors_dict, handle_col, protocol=pickle.HIGHEST_PROTOCOL)
handle_col.close()

## Genesets

### Curated

In [29]:
custom_gs_fp = "data/VRd Gene Sets from Literature.xlsx"

In [30]:
custom_gs = pd.read_excel(custom_gs_fp)
custom_gs = custom_gs.drop(labels=9, axis=0)
custom_gs = custom_gs.reset_index()

In [31]:
custom_gs.shape

(40, 15)

In [32]:
def gs_df_to_dict(pddf, label_col='display_label', gene_col='genes', descr_col = 'description', delim = ';'):
    gs_dict = {pddf[label_col][i]: {'description': pddf[descr_col][i],'features':pddf[gene_col][i].split(delim)} for i in range(pddf.shape[0])}
    return(gs_dict)

In [33]:
custom_gs_dict = gs_df_to_dict(custom_gs, label_col='display_label', gene_col='genes', delim = ';')

### Hallmark

In [34]:
# require import re
def fmt_gmt(gmt_fp):
    with open(gmt_fp) as gmt:
        gs_ls = gmt.readlines()
    elem = [x.split("\t") for x in gs_ls]
    gs_names = [x[0] for x in elem]
    gs_list = [x[2:] for x in elem]
    gs_list = [[re.sub("\n$", "", x) for x in gs] for gs in gs_list]
    gs_dict = dict(zip(gs_names, gs_list))
    return(gs_dict)

In [35]:
def add_description_gmt(gs_dict, description = None):
    names = [*gs_dict.keys()]
    gs_list = [*gs_dict.values()]
    if description is None:
        description_list = np.repeat(description, len(names))
    else:
        description_list = description
    res_dict = {names[i]: {'description':description_list[i], 'features':gs_list[i]} for i in range(len(names))}
    return res_dict

In [36]:
gs_description = pd.read_csv("./data/hallmark_gs_descriptions.csv")

In [37]:
desc_dict = dict(zip(gs_description['gs_name'], gs_description['gs_description']))

In [38]:
# read in all gene sets and merge into one master list
hallmark_fp = "./data/h.all.v7.5.1.symbols.gmt"
all_gs =  fmt_gmt(hallmark_fp)
desc_list = [desc_dict[x] for x in all_gs.keys()]
all_gs = add_description_gmt(all_gs, desc_list)

### Aggregate

In [39]:
all_gs.update(custom_gs_dict) # add custom geneset

In [40]:
all_gs.keys()

dict_keys(['HALLMARK_TNFA_SIGNALING_VIA_NFKB', 'HALLMARK_HYPOXIA', 'HALLMARK_CHOLESTEROL_HOMEOSTASIS', 'HALLMARK_MITOTIC_SPINDLE', 'HALLMARK_WNT_BETA_CATENIN_SIGNALING', 'HALLMARK_TGF_BETA_SIGNALING', 'HALLMARK_IL6_JAK_STAT3_SIGNALING', 'HALLMARK_DNA_REPAIR', 'HALLMARK_G2M_CHECKPOINT', 'HALLMARK_APOPTOSIS', 'HALLMARK_NOTCH_SIGNALING', 'HALLMARK_ADIPOGENESIS', 'HALLMARK_ESTROGEN_RESPONSE_EARLY', 'HALLMARK_ESTROGEN_RESPONSE_LATE', 'HALLMARK_ANDROGEN_RESPONSE', 'HALLMARK_MYOGENESIS', 'HALLMARK_PROTEIN_SECRETION', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 'HALLMARK_INTERFERON_GAMMA_RESPONSE', 'HALLMARK_APICAL_JUNCTION', 'HALLMARK_APICAL_SURFACE', 'HALLMARK_HEDGEHOG_SIGNALING', 'HALLMARK_COMPLEMENT', 'HALLMARK_UNFOLDED_PROTEIN_RESPONSE', 'HALLMARK_PI3K_AKT_MTOR_SIGNALING', 'HALLMARK_MTORC1_SIGNALING', 'HALLMARK_E2F_TARGETS', 'HALLMARK_MYC_TARGETS_V1', 'HALLMARK_MYC_TARGETS_V2', 'HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION', 'HALLMARK_INFLAMMATORY_RESPONSE', 'HALLMARK_XENOBIOTIC_METABOLISM', 'HAL

### Save pathway dictionary

In [41]:
handle_col = open('data/custom_gs_dict.pkl', 'wb')
pickle.dump(all_gs, handle_col, protocol=pickle.HIGHEST_PROTOCOL)
handle_col.close()

## Environment

In [42]:
session_info.show()